In [57]:
# if not installed, install
# !pip install tensorflow
# !pip install Keras==3.6.0

### Data Store

#### 1. prepare data/imagewoof2

#### 2. extract each of the ZIP files under the 'imagewoof2' directory and prepare the respective directories.

train https://drive.google.com/file/d/1rpTlk8osa-6zRk_akwvwlBGgICdnqqp-/view?usp=share_link 

val https://drive.google.com/file/d/1w33bjMpCuKgFw7XtlIl-uM3m9dQ3uRak/view?usp=share_link

### Test

In [8]:
import shutil
from pathlib import Path
from zipfile import ZipFile

from tensorflow.keras import Sequential
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Dense, Flatten, Lambda

from torch import argmax, jit, nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

2024-12-15 23:07:04.385644: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-15 23:07:04.576327: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734304024.598739   30480 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734304024.605552   30480 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 23:07:04.629258: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Train the Keras model

In [ ]:
input_image_size = (256, 256)
n_classes = 10

In [22]:
train_dir = 'data/imagewoof2/train'
valid_dir = 'data/imagewoof2/val'

In [23]:
# Load the tensorflow Dataset
train_ds = image_dataset_from_directory(train_dir, 
                                        seed=123, 
                                        shuffle=True,
                                        batch_size=32,
                                        image_size=input_image_size)
val_ds = image_dataset_from_directory(valid_dir,
                                      shuffle=True,
                                      batch_size=32,
                                      image_size=input_image_size)

Found 9025 files belonging to 10 classes.
Found 9025 files belonging to 10 classes.


In [24]:
from tensorflow import keras

data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
    ]
)

In [25]:
from tensorflow.keras.applications import (
    MobileNetV2,
    ResNet50,
    Xception,
    EfficientNetB0,
    mobilenet_v2,
    resnet,
    xception,
    efficientnet,
)

model_names = [MobileNetV2, ResNet50, Xception, EfficientNetB0]
preproc_names = [
    mobilenet_v2.preprocess_input,
    resnet.preprocess_input,
    xception.preprocess_input,
    efficientnet.preprocess_input,
]


# Fine tune model with last layer re-tuned on this dataset

In [28]:
import shutil
from pathlib import Path

for model_arch, pre_proc in zip(model_names, preproc_names):
    print(str(model_arch.__name__))
    
    base_model = model_arch(weights="imagenet", 
                            input_shape=(input_image_size[0], input_image_size[1], 3), 
                            include_top=False, classes=n_classes)
    
    base_model.trainable = False
    
    inputs = keras.Input(shape=(None, None, 3))
    resized =keras.layers.Resizing(input_image_size[0], input_image_size[1])(inputs)
    processed = pre_proc(resized)

    base = base_model(processed)
    outputs = Dense(n_classes, activation='softmax')(Flatten()(base))
    model = keras.Model(inputs=inputs, outputs=outputs)

    model.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_ds, epochs=2, validation_data=val_ds)

    # 保存形式を変更
    model_name = f'{model_arch.__name__}_keras_saved_model.keras'
    print(f'Saving model to {model_name}')
    model.save(model_name)

MobileNetV2


/tmp/ipykernel_30480/3874289574.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = model_arch(weights="imagenet",


Epoch 1/2
283/283 ━━━━━━━━━━━━━━━━━━━━ 242s 843ms/step - accuracy: 0.8788 - loss: 1.5217 - val_accuracy: 0.9717 - val_loss: 0.4251
Epoch 2/2
283/283 ━━━━━━━━━━━━━━━━━━━━ 237s 838ms/step - accuracy: 0.9753 - loss: 0.4247 - val_accuracy: 0.9849 - val_loss: 0.2666
Saving model to MobileNetV2_keras_saved_model.keras
ResNet50
Epoch 1/2
283/283 ━━━━━━━━━━━━━━━━━━━━ 924s 3s/step - accuracy: 0.7814 - loss: 7.0019 - val_accuracy: 0.9427 - val_loss: 1.8132
Epoch 2/2
283/283 ━━━━━━━━━━━━━━━━━━━━ 919s 3s/step - accuracy: 0.9641 - loss: 0.9824 - val_accuracy: 0.9825 - val_loss: 0.3952
Saving model to ResNet50_keras_saved_model.keras
Xception
Epoch 1/2
283/283 ━━━━━━━━━━━━━━━━━━━━ 1115s 4s/step - accuracy: 0.8934 - loss: 1.6019 - val_accuracy: 0.9631 - val_loss: 0.7773
Epoch 2/2
283/283 ━━━━━━━━━━━━━━━━━━━━ 1105s 4s/step - accuracy: 0.9614 - loss: 0.8114 - val_accuracy: 0.9803 - val_loss: 0.2588
Saving model to Xception_keras_saved_model.keras
EfficientNetB0
Epoch 1/2
283/283 ━━━━━━━━━━━━━━━━━━━━ 35

In [53]:
# prepare requirements.txt
!pip freeze | grep -E "tensorflow|keras|numpy" > requirements.txt

### ZIP ACTION

In [56]:
import zipfile

# select EfficientNetB0_keras_saved_model.keras
with zipfile.ZipFile('multiclass_model_package_image.zip', 'w') as zf:
    zf.write('EfficientNetB0_keras_saved_model.keras')
    zf.write('requirements.txt')
    zf.write('pred.py')